In [ ]:
import numpy as np
import cv2 as cv2
import os
import  xml.dom.minidom

In [ ]:
def get_image_sequence(dir):
    images = os.listdir(dir)
    return_list = []
    for img in images:
        img = os.path.join(dir, img)
        t = cv2.imread(img)
        return_list.append(t)
    return np.array(return_list)

def get_boxes_sequence(dir):
    dom = xml.dom.minidom.parse(dir)
    root = dom.documentElement
    frames = root.getElementsByTagName('frame')
    labels = []
    for frame in frames:
        frame_no = frame.getAttribute("number")
        objects = frame.getElementsByTagName('object')
        labels_curr = []
        for obj in objects:
            id = obj.getAttribute("id")
            # One object only have one box
            box = obj.getElementsByTagName("box")[0]
            xc = float(box.getAttribute("xc"))
            yc = float(box.getAttribute("yc"))
            w = float(box.getAttribute("w"))
            h = float(box.getAttribute("h"))
            x1 = xc - w / 2
            y1 = yc - h / 2
            x2 = xc + w / 2
            y2 = yc + h / 2
            # labels_curr.append(np.array([x, y, w, h, id]))
            labels_curr.append(np.array([x1, y1, x2, y2]))
        labels.append(np.array(labels_curr))
    return labels
    
def get_citypersons(dir='./data_PETS2009', type='train'):
    sub_folders = os.listdir(dir)
    train_img = []
    test_img = []
    train_label = []
    test_label = []
    for folder in sub_folders:
        sequence_path = os.path.join(dir, folder, 'View_001')
        xml_path = os.path.join(dir, folder, 'label.xml')
        images_sequence = get_image_sequence(sequence_path)
        labels_sequence = get_boxes_sequence(xml_path)
        if images_sequence.shape[0] != len(labels_sequence):
            print(sequence_path)
            print('# img = ' + str(images_sequence.shape[0]))
            print('# labels = ' + str(len(labels_sequence)))
            raise  Exception('# img != # labels')
        print('Load sequence: ' + str(images_sequence.shape))
        if folder == 'S2L1':
            # Use first 100 images for testing
            test_img.append(images_sequence[:100])
            test_label += labels_sequence[:100]
            train_img.append(images_sequence[100:])
            train_label += labels_sequence[100:]
        else:
            train_img.append(images_sequence)
            train_label += labels_sequence
    train_img = np.concatenate(train_img, axis=0)
    test_img = np.concatenate(test_img, axis=0)
    train_mean = np.mean(train_img, axis=(0, 1, 2))
    test_mean = np.mean(test_img, axis=(0, 1, 2))
    return train_img, train_label, train_mean, test_img, test_label, test_mean

In [ ]:
train_img, train_label, train_mean, test_img, test_label, test_mean = get_citypersons('../data_PETS2009/')

In [ ]:
import matplotlib.pyplot as plt
img = train_img[5]
boxes = train_label[5]
plt.imshow(img)

In [ ]:
for box in boxes:
    cv2.rectangle(img, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), [0, 0, 255], 2)
plt.imshow(img)

In [ ]:
print(np.mean(train_img, axis=(0, 1, 2)))